<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width=35% align=right>

# AI Agents & Automation — Chapter 14
## Safety, Alignment, and Governance

&copy; Dr. Yves J. Hilpisch<br>
AI-Powered by GPT-5.

### Overview

This notebook accompanies Chapter 14 — Safety Alignment Governance. It is self-contained and demonstrates the core ideas with small, readable code cells. Run cells from top to bottom; each code cell is preceded by a short explanation of what it does.


Minimal policy guard: allowlist + PII regex + time budget, wrapped around a tiny action.

In [ ]:
import re  # import regular expressions
import time  # import time utilities

# Compile a simple email pattern once at import time
EMAIL_PATTERN = r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}'
EMAIL = re.compile(EMAIL_PATTERN)  # pre-compiled email regex

def guard_call(
    tool,
    allow,
    max_ms,
    fn,
    **kwargs,
):
    # wrap tool invocation with guardrails
    """Gate a tool call by allowlist, PII scan, and latency budget.
    Returns a small dict with status/result.
    """
    start = time.perf_counter()  # start time

    # Enforce allowlist for tools
    if tool not in allow:  # enforce allowlist guard
        return {
            'status': 'error',  # surface policy violation
            'message': 'tool not allowed',  # explain rejection
        }

    # Block obvious email patterns in arguments
    if EMAIL.search(str(kwargs)):  # lightweight PII scan
        return {
            'status': 'error',  # surface policy violation
            'message': 'contains email; refuse or redact',  # block PII
        }

    # Execute the tool
    out = fn(**kwargs)  # call the tool
    ms = int((time.perf_counter() - start) * 1000)  # elapsed ms

    # Enforce latency budget
    if ms > max_ms:  # enforce latency budget
        return {
            'status': 'error',  # surface policy violation
            'message': f'over budget {ms} ms',  # explain throttle
        }

    # Return successful result
    return {'status': 'ok', 'latency_ms': ms, 'result': out}  # success payload
def echo(**kw):  # simple passthrough tool used for demo
    return {'echo': kw}  # echo payload for observability
    return {'echo': kw}
result = guard_call('echo', {'echo'}, 150, echo, text='hello')
print(result)  # invoke guard with safe input
print(guard_call('echo', {'echo'}, 150, echo, text='hello'))


<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width=35% align=right>